# Merging train data and ttf data & split by Lot_runnum

In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd

In [2]:
# data M01
# data_M02_train = pd.read_csv("../phm_etching_01M01-02/01_M01_DC_train.csv",
#                             encoding="utf8")

# data_M02_ttf = pd.read_csv("../phm_etching_01M01-02/01_M01_DC_train_ttf.csv",
#                             encoding="utf8")

# data M02
data_M02_train = pd.read_csv("../phm_etching_01M01-02/01_M02_DC_train.csv",
                            encoding="utf8")

data_M02_ttf = pd.read_csv("../phm_etching_01M01-02/01_M02_DC_train_ttf.csv",
                            encoding="utf8")

In [3]:
data_M02 = data_M02_train.merge(data_M02_ttf, left_on="time", right_on="time", how="left")
data_M02.drop_duplicates(inplace=True)

print(len(data_M02_train), len(data_M02_ttf), len(data_M02))
print(data_M02.columns)

5110542 5110542 5110317
Index(['time', 'Tool', 'stage', 'Lot', 'runnum', 'recipe', 'recipe_step',
       'IONGAUGEPRESSURE', 'ETCHBEAMVOLTAGE', 'ETCHBEAMCURRENT',
       'ETCHSUPPRESSORVOLTAGE', 'ETCHSUPPRESSORCURRENT', 'FLOWCOOLFLOWRATE',
       'FLOWCOOLPRESSURE', 'ETCHGASCHANNEL1READBACK', 'ETCHPBNGASREADBACK',
       'FIXTURETILTANGLE', 'ROTATIONSPEED', 'ACTUALROTATIONANGLE',
       'FIXTURESHUTTERPOSITION', 'ETCHSOURCEUSAGE', 'ETCHAUXSOURCETIMER',
       'ETCHAUX2SOURCETIMER', 'ACTUALSTEPDURATION',
       'TTF_FlowCool Pressure Dropped Below Limit',
       'TTF_Flowcool Pressure Too High Check Flowcool Pump',
       'TTF_Flowcool leak'],
      dtype='object')


In [4]:
from collections import Counter

temp_list = data_M02["recipe"].tolist()
temp_list.sort()

print(sorted(Counter(temp_list).items()))

[(66, 1443451), (67, 2716805), (68, 593630), (69, 218064), (70, 11731), (71, 40770), (72, 20474), (73, 33829), (74, 17062), (75, 1634), (76, 11793), (77, 1074)]


In [5]:
data_M02["Lot-runnum"] = data_M02["Lot"].apply(lambda x: str(x)) + "-" +\
data_M02["runnum"].apply(lambda x: str(x))
data_M02["recipe"].apply(lambda x: int(np.around(x, decimals=0)))
data_M02 = data_M02[data_M02["recipe"] == 67]

print(len(data_M02.loc[0, "Lot-runnum"]))
print(data_M02.loc[0, "Lot-runnum"])

13
9253-10511673


In [6]:
data_grouped  = data_M02.groupby(data_M02["Lot-runnum"])

# print(data_grouped.get_group("9253_10511673"))

for group in tqdm(data_grouped.groups.keys()):    
    data_export = data_grouped.get_group(group)
    
    # data M01
#     data_export.to_csv("../phm_etching_01M01-02/M01_Groups/Lot_runnum_" + str(group) + ".csv",
#                       index=None, encoding="utf8")
    
    # data M02
    data_export.to_csv("../phm_etching_01M01-02/M02_Groups_recipe_67/Lot_runnum_" + str(group) + ".csv",
                      index=None, encoding="utf8")

100%|██████████| 1670/1670 [00:55<00:00, 30.16it/s]


# Transform all data length to be same

* Reference:
  * https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.resample.html
  * https://docs.scipy.org/doc/scipy/reference/tutorial/interpolate.html
  

In [7]:
from scipy.interpolate import interp1d
from scipy import signal
from tqdm import tqdm
import codecs
import collections
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy

In [8]:
length_list = []
length_dict = {}

# dir_path = "../phm_etching_01M01-02/M01_Groups/"
dir_path = "../phm_etching_01M01-02/M02_Groups_recipe_67/"

for file in tqdm(os.listdir(dir_path)):
    data_temp = pd.read_csv(dir_path + file)
    length_list.append(len(data_temp))
    length_dict[file] = len(data_temp)

100%|██████████| 1670/1670 [00:22<00:00, 72.83it/s]


In [9]:
# find data length with highest frequency
# https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value

counter_dict = collections.Counter(length_list)
counter_dict =  dict(sorted(counter_dict.items(), key=lambda item: item[1], reverse=True))
most_frequent_length = list(counter_dict.keys())[0]
print("The most frequent length are:", list(counter_dict.keys())[0:5])

The most frequent length are: [1472, 1471, 1473, 1469, 1470]


In [10]:
# read in fault data for refactor

# fault_data = "../phm_etching_01M01-02/01_M01_train_fault_data.csv"
fault_data = "../phm_etching_01M01-02/01_M02_train_fault_data.csv"

data_fault = pd.read_csv(fault_data,
                        encoding="utf8")

fualt_time_list = data_fault.loc[:, "time"].tolist()
fault_name_list = data_fault.loc[:, "fault_name"].tolist()

In [13]:
# starting interpolation and downsampling the data
fault_time_inside_main_data = []

# dir_path = "../phm_etching_01M01-02/M01_Groups/"
# export_path = "../phm_etching_01M01-02/M01_Groups_same_length/TotalFault_"

dir_path = "../phm_etching_01M01-02/M02_Groups_recipe_67/"
export_path = "../phm_etching_01M01-02/M02_Groups_recipe_67_same_length/TotalFault_"


for fileIndex, file in tqdm(enumerate(os.listdir(dir_path))):
    situation_name = "Healthy"
    total_fault = 0
    
#     if fileIndex == 1:
#         fig, axs = plt.subplots(2, 1, figsize=(20, 5))
#         axs[0].plot(data_temp.loc[:, "recipe"])
#         axs[1].plot(data_export.loc[:, "recipe"])
#         plt.show()
#         break
    
    data_temp = pd.read_csv(dir_path + file)
    
    data_export = pd.DataFrame({
        "time": signal.resample(data_temp.loc[:, "time"], num=most_frequent_length),
        "Tool": [data_temp.loc[0, "Tool"] for i in range(most_frequent_length)],
        "Lot-runnum": [data_temp.loc[0, "Lot-runnum"] for i in range(most_frequent_length)]
    })
    
    # resampling for all columns
    Lot_runnum = str(data_temp.loc[0, "Lot-runnum"])
    for column in data_temp.columns:
        if column in ["time", "Tool", "Lot-runnum"]:
            continue
        data_export[column] = signal.resample(data_temp.loc[:, column], num=most_frequent_length)
        
    # check which fault type is the data
    for faultIndex, faultTime in enumerate(fualt_time_list):
        if faultTime in data_temp.loc[:, "time"].tolist():
            total_fault += 1
            fault_time_inside_main_data.append(faultTime)
            if situation_name == "Healthy":
                situation_name = fault_name_list[faultIndex].replace(" ", "")
            else:
                situation_name += ("-" + fault_name_list[faultIndex].replace(" ", ""))
            
    data_export.to_csv(export_path + str(total_fault)\
                       + "_" + situation_name + "_Lot_runnum_" + Lot_runnum +".csv",
                      index=None, encoding="utf8")
    
temp = fualt_time_list.copy()
for faultTime in fault_time_inside_main_data:
    temp.remove(faultTime)
    
print("Fault Time not inside main training data:", len(temp), temp)

1670it [02:24, 11.53it/s]

Fault Time not inside main training data: 79 [3915070, 3916822, 3918310, 3937494, 3940162, 3958954, 3968458, 8059958, 8105258, 8132488, 8136008, 9512452, 9518094, 12430304, 12435366, 12436028, 12437254, 12437984, 12439690, 12441326, 12514660, 12537920, 12580330, 12582892, 12588600, 12611078, 12619524, 12625938, 12635510, 12636486, 12649872, 12669900, 12681116, 12682450, 12684886, 12686968, 12688802, 12690282, 12692262, 12699926, 18057900, 18074648, 18097634, 18108796, 18139306, 18142434, 21905340, 22073942, 22075508, 22086924, 22779500, 22828180, 23082772, 23086028, 23087158, 23087648, 23348690, 23358554, 23630826, 23801808, 23834942, 25025806, 25637472, 28029182, 28113782, 28126778, 31427832, 35479832, 35672222, 36022710, 36132446, 36138198, 36145322, 36402870, 36575444, 36765886, 36767406, 36777452, 36797152]
